# Инсталляция библиотек

In [9]:
# Инсталляция билиотеки для технического анализа
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

  Using cached https://files.pythonhosted.org/packages/ac/cf/681911aa31e04ba171ab4d523a412f4a746e30d3eacb1738799d181e028b/TA-Lib-0.4.19.tar.gz
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437806 sha256=86bca06e0f3c4f949c8588cf8fa286cbf727605556017d488a721411fe70c1cf
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3

In [10]:
# Инсталляция библиотеки для отображения финансовых диаграмм
!pip install --upgrade mplfinance

Requirement already up-to-date: mplfinance in /usr/local/lib/python3.6/dist-packages (0.12.7a5)


In [11]:
!pip install yfinance

# Загрузка утилит и библиотек

In [12]:
# Библиотеки keras
from tensorflow.keras.models import Sequential, Model
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
#from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.metrics import sparse_categorical_accuracy, sparse_categorical_crossentropy
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization, Conv2D, SpatialDropout1D, Embedding, Flatten, Conv1D, LSTM, MaxPooling1D, concatenate, Input, GlobalMaxPooling1D, RepeatVector, SimpleRNN, GRU

# Библиотеки Scikit-learn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder

# Библиотеки для отображения
from matplotlib import pyplot as plt
# import seaborn as sns
# import statsmodels.tsa.api as smt
# import statsmodels.api as sm
from mplfinance.original_flavor import candlestick_ohlc

# Numpy & Pandas
import numpy as np
from pandas import pandas as pd, DataFrame as df

# System Libs
# import os
# from IPython.display import display
# import datetime
from __future__ import print_function
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets

# Библиотеки для загруprb данных
import importlib.util, sys, gdown


# Библиотека для технического анализа
import talib
import yfinance as yf

# Вспомогательные функции

## Функции для отображения диаграмм

In [13]:
def basic_diagram(fig_size=(22,10)):
  fig = plt.figure(figsize=fig_size)
  ax = fig.add_subplot(111)
  ax.grid()
  return ax


def add_lines(stock, ax, col_name, start=0, frame_len='All', color='red'):
  data = stock.copy()
  stop = (frame_len, data.shape[0])[int(frame_len=='All')]
  data = data.iloc[start: start + stop].reset_index(drop=True)

  ax.plot(data.index, data[col_name], color=color, label=col_name)
  return ax



def add_discrets(stock, ax, col_name, start=0, frame_len='All', color=['red', 'green']):
  data = stock.copy()
  stop = (frame_len, data.shape[0])[int(frame_len=='All')]
  data = data.iloc[start: start + stop].reset_index(drop=True)

  y_min, y_max = ax.get_ylim()
  delta = (y_max - y_min) * 0.1 / 2
  for idx, item in data[col_name].items():
    if item != 0:
      sign = (0, 1)[int(item > 0)]
      clr = color[sign]
      ax.vlines(idx, y_min + delta, y_max - delta, color=clr)
  return ax



def add_candles(stock, ax, start=0, frame_len='All'):
  data = stock.copy()
  stop = (frame_len, data.shape[0])[int(frame_len=='All')]
  data = data.iloc[start: start + stop].reset_index(drop=True)

  quotes = [tuple([
                idx,
                data['Open'].iloc[idx], 
                data['High'].iloc[idx], 
                data['Low'].iloc[idx], 
                data['Close'].iloc[idx]
                ]) for idx in data.index]
  
  candlestick_ohlc(ax, quotes, colordown='r', colorup='g', alpha=0.5)
  return ax


def add_ticks(stock, ax, col_name, start=0, frame_len='All', shift=5, color=['red', 'green']):
  data = stock.copy()
  stop = (frame_len, data.shape[0])[int(frame_len=='All')]
  data = data.iloc[start: start + stop].reset_index(drop=True)

  min = data[['High', 'Low']].min().min()
  max = data[['High', 'Low']].max().max()
  shift = (max - min) * (shift / 100)

  for idx, item in data[col_name].items():
    if item !=0:
      y = (data['High'].iloc[idx] + shift, data['Low'].iloc[idx] - shift)[int(item > 0)]
      sign = ('v', '^')[int(item > 0)]
      clr = color[int(item > 0)]
      ax.plot(idx, y, sign, color=clr)

  return ax


def show_diagram(legend=True):
  if legend:
    plt.legend()
  #plt.grid()
  plt.show()

# Определение стратегии торговли

In [14]:
# Сеть будет предсказывать какое действие лучше совершить покупку/продажу/никакое
# Стратегия строится на торговле на long и short сделках
#
# Для long сделок выделяется бюджет money, на который каждый раз при появлении сигнала на покупку совершается покупка n акций
# Все купленные акции продаются при первом же появлении сигнала к продаже. 
#
# Для short сделок определяется кредитный базис - это сумма, в пределах которой продаются акции брокера, если нет собственных 
# Купленные на short сделке акции продаются при следующем сигнале на продажу, в случае, если цена продажи стала ниже, чем была при покупке.
# В момент продажи акций по кредитной short сделке обязательно уменьшается кредитная поддержка, чтобы в случае, 
# если поведение рынка будет непредсказуемым не совершать сделки, которые будут ничем не обеспечены (бесконечный долг перед брокером).
# 
# При каждом сигнале на продажу продаются все собственные акции, а в случае выгодной для сделки по кредитным акциям 
# выполняется проверка аккумулятора денег (money) и если сумма средств от сделки позволяет, то восстанавливается уровень кредитной поддержки до базового или до стартового (макс уровень)
#

def trading_strategy(tr_data, money=50000, shares=0, basic_loan_security=50000, action_col='action', bear_margin_wait=1.1, bull_margin_wait=1.3, fix_margin=21, stop_loss_steps=21, full_history=False, verbose=False, only_actions=True):
  actions = {0: 'none', 1: 'buy', -1:'sell'}
  loan_security = basic_loan_security
  loan_shares = 0
  loan_shares_price = 0
  action_done = 0
  stop_loss = 0
  margin_steps = 0
  shares_price = 0

  money_history = df(columns=['action', 'money', 'own_shares', 'cur_price', 'loan_security', 'loan_shares', 'loan_shares_price'])

  items = tr_data[[action_col, 'Close']].values

  for item in items:
    action, price = item[0], item[1]
    
    if action == 1 and money > price:
      shares += money // price # Покупаем на все имеющиеся деньги акции (покупаем целое количество)
      money = money % price # Считаем сколько осталось денег после покупки акций
      action_done = 1
      shares_price = price

    # Если должны продавать
    elif action == -1:
      if shares > 0:
        if shares_price * bull_margin_wait <= price or margin_steps > fix_margin:
          # Продаем все акции и увеличиваем капитал на заработанные деньги
          # Количество акцией сбрасываем в 0
          action_done = -1
          sell_value = shares * price 
          shares = 0
          money += sell_value       

      if loan_shares > 0:
        if loan_shares_price >= price * bear_margin_wait or stop_loss > stop_loss_steps:
          loan_shares = 0
          sell_value = loan_shares * (loan_shares_price - price)
          money += sell_value
          loan_security += money * 0.5
          money *= 0.5
          stop_loss = 0

      elif loan_shares == 0 and loan_security >= price:
        loan_shares = loan_security // price 
        sell_value = loan_shares * price
        loan_shares_price = price
        money += sell_value
        loan_security -= sell_value

    else:
      action = 0
      action_done = 0
      if not only_actions:
        if stop_loss > stop_loss_steps and loan_shares > 0:
          loan_shares = 0
          sell_value = loan_shares * (loan_shares_price - price)
          money += sell_value
          loan_security += money * 0.5
          money *= 0.5
          stop_loss = 0
        if margin_steps > fix_margin and shares > 0:
          action_done = -1
          sell_value = shares * price 
          shares = 0
          money += sell_value

    stop_loss += 1
    margin_steps += 1

    money_history = money_history.append(
        {'action': actions[action], 
         'action_done': action_done,
         'money': round(money, 2), 
         'own_shares': shares, 
         'shares_price': shares_price,
         'current_price': price, 
         'basic_loan_security': basic_loan_security, 
         'loan_security': loan_security, 
         'loan_shares': loan_shares, 
         'loan_shares_price': loan_shares_price,
         'total_assets': round(money + loan_security + shares * price + (loan_shares * loan_shares_price - loan_shares * price),2),
         'stop_loss': stop_loss,
         'margin_steps': margin_steps
        }, 
        ignore_index=True)

  if verbose:
    print(money_history)

  return (money_history.iloc[-1], money_history)[int(full_history)]

# Загрузка данных для обучения, разбиение на датасеты

In [25]:
def get_competition():
  # Загружаем обучающий и проверочный датасет (конкурсный)
  path = '/content/drive/MyDrive/Colab Notebooks/Конкурс Трейдинг/Котировки'
  train_dataset = pd.read_csv(f'{path}/train_dataset.csv', sep=",")
  val_dataset = pd.read_csv(f'{path}/val_dataset.csv', sep=",")  
  test_dataset = pd.read_csv(f'{path}/test_dataset.csv', sep=",")
  
  # Здесь указать тестовый датасет для проверки работы сети и снять комментарий
  #test_dataset = pd.read_csv(f'{path}/test_dataset.csv', sep=",")  
  #
  #data = yf.download('TSLA','2020-09-25','2021-01-25',interval='1h')
  #test_dataset=pd.DataFrame(data.reset_index(),columns=['Open', 'High', 'Low', 'Close','Volume'])

  dataset_ = train_dataset.copy()
  dataset_ = pd.concat([dataset_, val_dataset.copy()], ignore_index=True)
  
  #Снять комментарий для присожинения тестового датасета
  dataset_ = pd.concat([dataset_, val_dataset.copy()], ignore_index=True)
  return dataset_

In [26]:
# для проведения экспериментов удаляем data из памяти colab и создаем заново
try:
  del data
except:
  print('First time create dataset')

# Загружаем данные для обучения и проверки
data = get_competition().copy() 

open, close, high, low = data['Open'], data['Close'], data['High'], data['Low']

# Выполняем необходимые преобразования
steps = [3, 4, 5, 6, 9, 12, 21]

def compare(a, b):
  res = 0
  if a < b:
    res = -1
  elif a > b:
    res = 1
  return res

def trade_action(a, b):
  actions = {'none': 0, 'buy': 1, 'sell': -1}
  action = 'none'
  if a < b:
    action = 'buy'
  elif b < a:
    action = 'sell'
  return actions[action]

for step in steps:
  data[f'sma_{step}'] = talib.EMA(close, step)
  for shift in range(len(steps)):
    data[f'sma_{step}_sh{shift}'] = data[f'sma_{step}'].shift(shift)
    data[f'sma_{step}_cur_prev{shift}'] = data.apply(lambda x: compare(x[f'sma_{step}'], x[f'sma_{step}_sh{shift}']), axis=1)

  data[f'cci_{step}'] = talib.CCI(high, low, close, step)
  for shift in range(len(steps)):
    data[f'cci_{step}_sh{shift}'] = data[f'sma_{step}'].shift(shift)
    data[f'cci_{step}_cur_prev{shift}'] = data.apply(lambda x: compare(x[f'cci_{step}'], x[f'cci_{step}_sh{shift}']), axis=1)

  data[f'cci_{step}_bm1'] = data.apply(lambda x: (0, 1)[int(x[f'cci_{step}'] >= 100)], axis=1)
  data[f'cci_{step}_bm2'] = data.apply(lambda x: (0, 1)[int(x[f'cci_{step}'] >= 200)], axis=1)
  data[f'cci_{step}_bm3'] = data.apply(lambda x: (0, 1)[int(x[f'cci_{step}'] <= -100)], axis=1)
  data[f'cci_{step}_bm4'] = data.apply(lambda x: (0, 1)[int(x[f'cci_{step}'] <= -200)], axis=1) 


# Индикаторы
ext_cdl = []
cdl = [item for item in talib.__TA_FUNCTION_NAMES__ if 'CDL' in item]
for item in cdl:
  col = getattr(talib, item)(open, high, low, close)
  if col[col != 0].count() >= data.shape[0] * 0.1:
    data[item.lower()] = col / 100
  else:
    ext_cdl.append(item)

cdl = [item for item in cdl if item not in ext_cdl]

# Критерий для выполненя торговых операций - пробой более быстрой sma(ema) менее быструю 
# Снизу-вверх - сигнал к покупке
# Сверзу-вниз - сигнал к продаже

for step1 in steps:
  for step2 in steps:  
    if step1 == step2:
      continue
    data[f'sma_{step1}<sma_{step2}'] = data.apply(lambda x: (0, 1)[int(x[f'sma_{step1}'] < x[f'sma_{step2}'])], axis=1)
    data[f'sma_{step1}<sma_{step2}_sh1'] = data[f'sma_{step1}<sma_{step2}'].shift(1)
    data[f'action_{step1}_{step2}'] = data.apply(lambda x: trade_action(
                          x[f'sma_{step1}<sma_{step2}'],
                          x[f'sma_{step1}<sma_{step2}_sh1']
                        ), 
                      axis=1)

step1, step2 = 3, 5
data['action'] = data[f'action_{step1}_{step2}']

prev_act = 0
for idx in data.index:
  data.loc[idx, ['action_prev']] = prev_act
  act = data['action'].iloc[idx]
  prev_act = (prev_act, act)[int(act != 0)]


# Удаляем колонки с пустыми и с абсолютными значениями
# Остаются только относительные признаки, которые должна научиться выделять сеть
data.dropna(axis=0, inplace=True)
dataset = data.copy()
dataset.drop(['Open', 'High', 'Low', 'Volume', 'Date'], axis=1, inplace=True)
dataset.drop(['Close'], axis=1, inplace=True)
for step in steps:
  dataset.drop([f'sma_{step}'], axis=1, inplace=True)
  dataset.drop([f'cci_{step}'], axis=1, inplace=True)
  for shift in range(len(steps)):
    dataset.drop([f'sma_{step}_sh{shift}'], axis=1, inplace=True)
    dataset.drop([f'cci_{step}_sh{shift}'], axis=1, inplace=True)

In [19]:
dataX = yf.download('TSLA','2020-09-25','2021-01-25',interval='1h')
X_test=pd.DataFrame(dataX.reset_index(),columns=['Open', 'High', 'Low', 'Close','Volume'])
X_test.shape

[*********************100%***********************]  1 of 1 completed


(568, 5)

**Подготовка датасетов**

In [20]:
# Разделяем датасет на тренировочный, валидационный и тестовый 
# Определяем глубину данных для обучения (количество предыдущих шагов)

xLen = 100

##########################################################################
# Здесь нужно скорректировать границы валидационного и тестового датасетов
# Размер тренировочного датасета выяичсляется автоматически
#
##########################################################################
val_length = 300
test_length = 568 # <---- Исправить на размер тестовой выборки

# После удаления строк датасета, содержащих NAN определяем длину тренировочного датасета
train_length = dataset.shape[0] - val_length - test_length

# [start : stop] - указываются границы датасетов
edges = {'train': [0, train_length], 
         'val': [train_length - xLen, train_length + val_length], 
         'test': [train_length + val_length - xLen, train_length + val_length + test_length]}
  
source_dsets = []
dsets = []

for dset in edges:
  start = edges[dset][0] 
  stop = edges[dset][1]
  source_dsets.append(data.iloc[start : stop])
  dsets.append(dataset.iloc[start : stop].copy())

s_train, s_val, s_test = source_dsets
train, val, test = dsets

train.shape, val.shape, test.shape

((2406, 264), (400, 264), (668, 264))

In [21]:
# Здесь можно попробовать поменять индикатор, на основании которого осуществляется торговля
# По умолчанию - это значение action - рассчитанное на основании смены трендов (cross ema с разными скоростями)
# К этим значениям применяется торговая стратегия и вычисляется результат (т.е. - это целевой результат, к которому стремится сеть)
# Используя выпадающие списки можно вместо cross ema попробовать ориентироваться на другие индикаторы-паттерны, которые встречаются в выборке достаточно часто
# А также проверить стратегию на тренировочном/валидационном и тестовом датасетах, т.е. получить true значения 

@interact(marker=['action', *cdl], dset=['test', 'val', 'train'])
def show_markers(marker, dset):
  source_dset = dict(zip(['train', 'val', 'test'], source_dsets))[dset]
  
  # Параметры торговой стратегии 
  params = {'money': 50000, 'basic_loan_security':50000, 'bear_margin_wait': 1.03, 'bull_margin_wait': 1.3, 'fix_margin':21, 'stop_loss_steps':21, 'verbose':False}

  money = trading_strategy(source_dset, action_col=marker.lower(), **params, full_history=True)
  money['total_money'] = money['money'] + money['loan_security']
  money['High'] = money['money']
  money['Low'] = money['money']
  money['action_id'] = money['action'].map({'none': 0, 'buy': 1, 'sell': -1})

  print(f'The possible trading result: {money["total_assets"].iloc[-1]}')
  ax = basic_diagram(fig_size=(22,8))
  ax = add_candles(source_dset, ax)
  ax = add_ticks(source_dset, ax, 'action', shift=3)
  ax = add_ticks(source_dset, ax, marker.lower(), shift=5, color=['magenta', 'blue'])

  ax1 = basic_diagram(fig_size=(22,4))
  ax1 = add_lines(money, ax1, 'money')
  ax1 = add_lines(money, ax1, 'loan_security', color='blue')
  ax1 = add_lines(money, ax1, 'total_money', color='green')

  ax1 = add_ticks(money, ax1, 'action_id', shift=5, color=['red', 'magenta'])
  
  ax2 = basic_diagram(fig_size=(22,3))
  ax2 = add_lines(money, ax2, 'own_shares')
  ax2 = add_lines(money, ax2, 'loan_shares', color='blue')
  ax2 = add_discrets(money, ax2, 'action_done', color=['grey', 'green'])

  show_diagram()
  money.to_csv('/content/drive/MyDrive/Colab Notebooks/Конкурс Трейдинг/res.csv', sep=';')

interactive(children=(Dropdown(description='marker', options=('action', 'CDLBELTHOLD', 'CDLCLOSINGMARUBOZU', '…

In [22]:
# создание таймсерий для обучения сети
batch_size = 30

# Категории - трейдинговые активности {'none': 0, 'buy': 1, 'sell': -1}
cat = OneHotEncoder(sparse=False)
cat.fit(np.array([0,1,-1]).reshape(-1,1))

action_col = 'action'

x_train = train
y_train = cat.transform(train.loc[:, action_col].to_numpy().reshape(-1,1))

x_val = val
y_val = cat.transform(val.loc[:, action_col].to_numpy().reshape(-1,1))

x_test = test  
y_test = cat.transform(test.loc[:, action_col].to_numpy().reshape(-1,1))

xScaler = RobustScaler().fit(x_train)
x_train = xScaler.transform(x_train)
x_test = xScaler.transform(x_test)
x_val = xScaler.transform(x_val)

trainDataGen = TimeseriesGenerator(x_train, y_train, length=xLen, stride=1, sampling_rate=1, batch_size=batch_size) 
valDataGen = TimeseriesGenerator(x_val, y_val, length=xLen, stride=1, sampling_rate=1, batch_size=batch_size)
testDataGen = TimeseriesGenerator(x_test, y_test, length=xLen, stride=1, sampling_rate=1, batch_size=batch_size)

# Сеть

In [ ]:
def LSTM_model(xLen, channels):
  input = Input((xLen, channels), name="Input")
  x = LSTM(100)(input)
  x = RepeatVector(xLen)(x)
  x = LSTM(100)(x)
  x = RepeatVector(xLen)(x)
  x = Conv1D(30, (2))(x)
  x = MaxPooling1D()(x)
  x = Flatten()(x)
  x = Dense(100)(x)
  x = Dropout(0.3)(x)
  x = Dense(10)(x)
  x = Dropout(0.3)(x)
  x = Dense(3, activation='softmax')(x)
  return Model(input, x)

# Обучение

In [ ]:
# Callback для управления процессом обучения
# Callback проверяет и сохраняет лучшую модель с точки зрения выполнения стратегии best_model_{model_name}.h5 и модель с последней эпохи model_{model_name}
#
class BestTradeCallback(Callback):
  def __init__(self, epochs, trading_params={}, model_name='model', best_trading=True, loss_n_acc=True, save=True, data_set='test', patience=5):
    super(BestTradeCallback, self).__init__()
    self.best_trading_res = 0
    self.best_model_epoch = 0
    self.val_loss = 0
    self.epochs = epochs
    self.categorical_crossentropy = 0
    self.trading_params = trading_params
    self.data_set = data_set
    self.save = save
    self.model_name = f'{model_name}'
    self.loss_n_acc = loss_n_acc
    self.best_trading = best_trading
    self.weights_path = f'/content/drive/MyDrive/Colab Notebooks/Конкурс Трейдинг/models/'
    self.model_path = f'/content/drive/MyDrive/Colab Notebooks/Конкурс Трейдинг/models/'
    self.time_series = dict(zip(['train', 'val', 'test'], [trainDataGen, valDataGen, testDataGen]))[data_set]
    self.source_dset = dict(zip(['train', 'val', 'test'], source_dsets))[data_set]
    self.patience = patience
    self.plateau = 0

  def change_lr(self, epoch):
    lr_schedule = [0.01, 0.005, 0.001, 0.0005, 0.0001]
    lr = round(K.get_value(model.optimizer.lr) * 0.8, 6)
    K.set_value(model.optimizer.lr, lr)
    print(f'(epoch: {epoch}). Set lr = {lr}')

  def on_epoch_end(self, epoch, logs=None):
    if epoch == 0:
      self.val_loss = logs['loss']
      self.val_categorical_crossentropy = logs['categorical_crossentropy']
    
    if self.best_trading:
      res = self.predict()

      if self.best_trading_res < res or (self.best_trading_res == res and logs['loss'] < self.val_loss):
        self.val_loss = logs['loss']
        self.val_categorical_crossentropy = logs['categorical_crossentropy']
        self.best_model_epoch = epoch
        self.best_trading_res = res    
        self.save_weights(f'best_weights_{self.model_name}.h5') 
      else:
        self.plateau += 1

    elif self.loss_n_acc:
      if logs['loss'] < self.val_loss:
        self.val_loss = logs['loss']
        self.val_categorical_crossentropy = logs['categorical_crossentropy']
        self.best_model_epoch = epoch
        res = self.predict()
        self.best_trading_res = res
        self.save_weights(f'best_weights_{self.model_name}.h5')
        
    print(f'(End epoch: {epoch}). {logs}')
    print(f'(End epoch: {epoch}). Best result on {self.best_model_epoch} epoch: {self.best_trading_res}, model_loss: {round(self.val_loss,2)}, model_categorical_crossentropy: {round(self.val_categorical_crossentropy,2)}')
    if self.plateau > self.patience:
      #self.load_weights()
      self.change_lr(epoch)
      self.plateau = 0

  def on_train_end(self, logs=None):
    print('The best result:', self.best_trading_res)
    self.save_model(f'{self.model_name}.h5')
    self.load_weights(f'best_weights_{self.model_name}.h5')
    self.save_model(f'best_{self.model_name}.h5')
      
  def predict(self):
    pred = model.predict(self.time_series)
    test_d = self.source_dset.iloc[xLen:].copy()
    test_d['action_pred'] = cat.inverse_transform(pred)
    return trading_strategy(test_d, action_col='action_pred', **self.trading_params)['total_assets']

  def load_weights(self, model_name):
    global model
    model.load_weights(f'{self.weights_path}{model_name}')

  def save_weights(self, model_name):
    model.save_weights(f'{self.weights_path}{model_name}', overwrite=True, save_format='h5', options=None)

  def save_model(self, model_name):
    model.save(f'{self.model_path}{model_name}')

In [ ]:
def train_net(model_name):
  # Обучение сети
  global model, trading_params
  epochs = 70
  channels = trainDataGen[0][0].shape[2]
  model = LSTM_model(xLen, channels)
  model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-2), metrics=['categorical_crossentropy'])

  # торговая стратегия
  trading_params = {'money': 50000, 'basic_loan_security':50000, 'bear_margin_wait': 1.3, 'bull_margin_wait': 1.3, 'fix_margin':49, 'stop_loss_steps':49, 'verbose':False}
  history = model.fit(
                      trainDataGen, 
                      epochs=epochs, 
                      verbose=0, 
                      callbacks=[BestTradeCallback(epochs, trading_params, best_trading=True, loss_n_acc=True, model_name=model_name)],
                      validation_data = valDataGen
                      )

try:
  del model
  print('Model is deleted')
except:
  print("Can't find model to delete")


# Запус обучения модели с выбором типа модели и с указанием названия модели
train_net('LSTM_01')

# Проверка

In [23]:
# Загрузка модели для проверки
model_name = 'best_LSTM_01'
model = load_model(f'/content/drive/MyDrive/Colab Notebooks/Конкурс Трейдинг/models/{model_name}.h5')

In [27]:
# Интерактивная проверка модели
# При выполяннении сохраняется так же отчет о торговой стратегии
@interact(dset=['test', 'val', 'train'], show_true=[False, True])
def eval_pred(dset, show_true):
  source_dset = dict(zip(['train', 'val', 'test'], source_dsets))[dset]
  time_series = dict(zip(['train', 'val', 'test'], [trainDataGen, valDataGen, testDataGen]))[dset]

  pred = model.predict(time_series)
  test_d = source_dset.iloc[xLen:].copy()

  test_d['action_pred'] = cat.inverse_transform(pred)

  params = {'money': 50000, 'basic_loan_security':50000, 'bear_margin_wait': 1.5, 'bull_margin_wait': 1.5, 'fix_margin':49, 'stop_loss_steps':49, 'verbose': False, 'only_actions': True}
  #params = trading_params
  money = trading_strategy(test_d, action_col='action_pred', full_history=True, **params)
  money['total_money'] = money['money'] + money['loan_security']
  money['High'] = money['money']
  money['Low'] = money['money']
  money['action_id'] = money['action'].map({'none': 0, 'buy': 1, 'sell': -1})

  print(f'Trading strategy (true): ', trading_strategy(test_d[['Close', 'action']], full_history=False,  **params)['total_assets'])
  print(f'Traiding strategy (prediction): ', money['total_assets'].iloc[-1])

  ax = basic_diagram(fig_size=(22,6))
  ax = add_candles(test_d, ax)
  ax = add_ticks(test_d, ax, 'action_pred', shift=5, color=['magenta', 'blue'])
  if show_true:
    ax = add_ticks(test_d, ax, 'action', shift=3)

  ax1 = basic_diagram(fig_size=(22,3))
  ax1 = add_lines(money, ax1, 'money')
  ax1 = add_lines(money, ax1, 'loan_security', color='blue')
  ax1 = add_lines(money, ax1, 'total_assets', color='green')
  ax1 = add_ticks(money, ax1, 'action_id', shift=6, color=['red', 'magenta'])
  

  ax2 = basic_diagram(fig_size=(22,3))
  ax2 = add_lines(money, ax2, 'own_shares')
  ax2 = add_lines(money, ax2, 'loan_shares', color='blue')
  ax2 = add_discrets(money, ax2, 'action_done', color=['grey', 'green'])

  show_diagram()

  # Сохранение отчета торговой стратегии
  money.to_csv(f'/content/drive/MyDrive/Colab Notebooks/Конкурс Трейдинг/res_{model_name}_{dset}.csv', sep=';')

interactive(children=(Dropdown(description='dset', options=('test', 'val', 'train'), value='test'), Dropdown(d…